In [3]:
from typing import Dict, List, Callable, Literal, Optional
import enum
import heapq
import numpy as np

### Task 1

In [4]:
type Components = Dict[str, Literal[0, 1]]

class Environment:
    def __init__(self):
        self.__components: Components = \
            dict([(chr(ord('A') + i), np.random.randint(0, 2)) for i in range(9)])
        
    def display_state(self):
        print(self.__components)
    
    def get_percept(self) -> Components:
        return self.__components
    
    def patch(self, component: str):
        self.__components.update({component: 0})

class Agent:
    def __init__(self):
        self.__patches: List[str] = []
    
    def scan(self, components: Components):
        for component in components:
            if components[component] == 0:
                print(f'<success>: {component} is secure.')
                continue
            print(f'<warning>: {component} is not secure!')
            self.__patches.append(component)

    def act(self, patch: Callable[[str], None]):
        for component in self.__patches:
            patch(component)
        self.__patches.clear()

def run_agent():
    env = Environment()
    agent = Agent()

    env.display_state()
    agent.scan(env.get_percept())
    agent.act(env.patch)
    env.display_state()

run_agent()

{'A': 1, 'B': 0, 'C': 1, 'D': 1, 'E': 1, 'F': 1, 'G': 1, 'H': 0, 'I': 1}
<warning>: A is not secure!
<success>: B is secure.
<warning>: C is not secure!
<warning>: D is not secure!
<warning>: E is not secure!
<warning>: F is not secure!
<warning>: G is not secure!
<success>: H is secure.
<warning>: I is not secure!
{'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 0, 'I': 0}


### Task 2

In [5]:
class States(enum.Enum):
    UNDERLOADED = 'UNDERLOADED'
    BALANCED = 'BALANCED'
    OVERLOADED = 'OVERLOADED'

BALANCED_COUNT = 10

class Server:
    def __init__(self, initial_tasks: List[int] = []):
        self.__tasks: List[int] = []
        self.__state = States.UNDERLOADED
        for task in initial_tasks:
            self.add(task)
    
    def add(self, task: int):
        self.__tasks.append(task)
        self.__update_state()

    def remove(self) -> int:
        task: int = self.__tasks.pop()
        self.__update_state()
        return task

    def get_state(self) -> States:
        return self.__state

    def __str__(self):
        return f'status: {self.__state.value:<11} | tasks[{len(self.__tasks)}]: {', '.join(map(str, self.__tasks))}'
    
    def __gt__(self, obj):
        return len(self.__tasks) < len(obj.__tasks)
    
    def __len__(self):
        return len(self.__tasks)
    
    def __update_state(self):
        n = len(self.__tasks)
        if n > BALANCED_COUNT:
            self.__state = States.OVERLOADED
        elif n == BALANCED_COUNT:
            self.__state = States.BALANCED
        else:
            self.__state = States.UNDERLOADED

class Environment:
    def __init__(self):
        self.__servers: List[Server] = [
            Server(np.random.randint(0, 20, np.random.randint(15))) for _ in range(5)
        ]

    def display_state(self):
        total = 0
        for i, server in enumerate(self.__servers):
            print(f'{i}:', server)
            total += len(server)
        print('Total tasks:', total)
    
    def update_servers(self, servers: List[Server]):
        self.__servers = servers

    def get_precept(self) -> List[Server]:
        return self.__servers

class Agent:
    class ServerWrapper:
        def __init__(self, server: Server):
            self.server = server
        def __gt__(self, obj):
            return len(self.server) > len(obj.server)

    def __init__(self):
        self.__balanced: List[Server] = []
        self.__overloaded_pq: List[Server] = []
        self.__underloaded_pq: List[Agent.ServerWrapper] = []

    def scan(self, servers: List[Server]):
        for server in servers:
            if server.get_state() == States.OVERLOADED:
                heapq.heappush(self.__overloaded_pq, server)
            elif server.get_state() == States.UNDERLOADED:
                heapq.heappush(self.__underloaded_pq, Agent.ServerWrapper(server))
            else:
                self.__balanced.append(server)
    
    def act(self):
        while self.__overloaded_pq and self.__underloaded_pq:
            overloaded: Server = heapq.heappop(self.__overloaded_pq)
            underloaded: Agent.ServerWrapper = heapq.heappop(self.__underloaded_pq)

            while len(overloaded) > BALANCED_COUNT and len(underloaded.server) < BALANCED_COUNT:
                underloaded.server.add(overloaded.remove())

            if overloaded.get_state() != States.BALANCED:
                heapq.heappush(self.__overloaded_pq, overloaded)
            else:
                self.__balanced.append(overloaded)

            if underloaded.server.get_state() != States.BALANCED:
                heapq.heappush(self.__underloaded_pq, underloaded)
            else:
                self.__balanced.append(underloaded.server)
        
        return self.__balanced + self.__overloaded_pq + list(map(lambda s: s.server, self.__underloaded_pq))

def run_agent():
    env = Environment()
    agent = Agent()
    
    env.display_state()
    agent.scan(env.get_precept())
    env.update_servers(agent.act())
    env.display_state()

run_agent()

0: status: BALANCED    | tasks[10]: 3, 0, 16, 2, 13, 4, 8, 11, 4, 17
1: status: OVERLOADED  | tasks[11]: 13, 11, 13, 7, 6, 2, 6, 4, 8, 0, 17
2: status: OVERLOADED  | tasks[12]: 4, 9, 9, 0, 16, 15, 4, 4, 9, 15, 9, 19
3: status: UNDERLOADED | tasks[3]: 3, 9, 10
4: status: BALANCED    | tasks[10]: 2, 1, 16, 11, 11, 1, 4, 1, 3, 16
Total tasks: 46
0: status: BALANCED    | tasks[10]: 3, 0, 16, 2, 13, 4, 8, 11, 4, 17
1: status: BALANCED    | tasks[10]: 2, 1, 16, 11, 11, 1, 4, 1, 3, 16
2: status: BALANCED    | tasks[10]: 4, 9, 9, 0, 16, 15, 4, 4, 9, 15
3: status: BALANCED    | tasks[10]: 13, 11, 13, 7, 6, 2, 6, 4, 8, 0
4: status: UNDERLOADED | tasks[6]: 3, 9, 10, 19, 9, 17
Total tasks: 46


### Task 3

In [6]:
class Backup(enum.Enum):
  COMPLETED = 'Completed'
  FAILED = 'Failed'

choices = list(Backup)

class Environment:
  def __init__(self, backup_counts: int):
    self.__backups: List[Backup] = [np.random.choice(choices) for _ in range(backup_counts)]
  
  def display_state(self):
    print(*map(lambda obj: (obj[0], obj[1].value), enumerate(self.__backups)), sep='\n')

  def get_precept(self) -> List[Backup]:
    return self.__backups

  def retry_backup(self, idx: int):
    self.__backups[idx] = Backup.COMPLETED
  
class Agent:
  def __init__(self):
    self.__failed: List[int] = []
  
  def scan(self, backups: List[Backup]):
    self.__failed = [i for i, backup in enumerate(backups) if backup == Backup.FAILED]
    print(self.__failed)
  
  def act(self, retry_backup: Callable[[int], None]):
    for i in self.__failed: retry_backup(i)

def run_agent():
  env = Environment(10)
  agent = Agent()

  env.display_state()
  agent.scan(env.get_precept())
  agent.act(env.retry_backup)
  env.display_state()

run_agent()

(0, 'Completed')
(1, 'Failed')
(2, 'Completed')
(3, 'Failed')
(4, 'Failed')
(5, 'Completed')
(6, 'Completed')
(7, 'Failed')
(8, 'Completed')
(9, 'Failed')
[1, 3, 4, 7, 9]
(0, 'Completed')
(1, 'Completed')
(2, 'Completed')
(3, 'Completed')
(4, 'Completed')
(5, 'Completed')
(6, 'Completed')
(7, 'Completed')
(8, 'Completed')
(9, 'Completed')


### Task 4

In [7]:
class Status(enum.Enum):
  SAFE = 'Safe'
  LOW = 'Low Risk Vulnerable'
  HIGH = 'High Risk Vulnerable'

choices = list(Status)

class Environment:
  def __init__(self):
    self.__components: Dict[str, Status] = {chr(ord('A') + i): np.random.choice(choices) for i in range(9)}
  
  def display_state(self):
    print(*map(lambda obj: f'{obj[0]}: {obj[1].value}', self.__components.items()), sep='\n')

  def get_precept(self) -> Dict[str, Status]:
    return self.__components
  
  def patch(self, component: str):
    self.__components.update({component: Status.SAFE})

class Agent:
  def __init__(self):
    self.__priority = {
      Status.SAFE: 0,
      Status.LOW: 3, # priority value
      Status.HIGH: 0
    }
    self.__vulnerable: Dict[str, Status] = {}

  def utility(self, status: Status):
    return self.__priority.get(status, 0)
  
  def scan(self, components: Dict[str, Status]):
    for component, status in components.items():
      if status != Status.SAFE:
        print(f'Warning: {component} is {status.value}')
        if status == Status.HIGH:
          print('Premium service needed to patch.')
        else:
          self.__vulnerable.update({component: status})
      else:
        print(f'Success: {component} is {status.value}')

  def choose_action(self) -> Optional[str]:
    max_component = max(self.__vulnerable, key=lambda key: self.utility(self.__vulnerable.get(key)), default=None)
    self.__vulnerable.pop(max_component, None)
    return max_component

def run_agent():
  env = Environment()
  agent = Agent()

  env.display_state()
  components = env.get_precept()
  agent.scan(components)
  while True:
    component = agent.choose_action()
    if not component:
      break
    env.patch(component)
  env.display_state()

run_agent()

A: Safe
B: High Risk Vulnerable
C: High Risk Vulnerable
D: Low Risk Vulnerable
E: Safe
F: High Risk Vulnerable
G: Safe
H: Low Risk Vulnerable
I: High Risk Vulnerable
Success: A is Safe
Premium service needed to patch.
Premium service needed to patch.
Success: E is Safe
Premium service needed to patch.
Success: G is Safe
Premium service needed to patch.
A: Safe
B: High Risk Vulnerable
C: High Risk Vulnerable
D: Safe
E: Safe
F: High Risk Vulnerable
G: Safe
H: Safe
I: High Risk Vulnerable


### Task 5

In [54]:
from typing import Set

class Task:
  def __init__(self, room: str, medicine: str, patient_id: int, schedule: int):
    self.room = room
    self.medicine = medicine
    self.patient_id = patient_id
    self.schedule = schedule

  def __lt__(self, obj) -> bool:
    return self.schedule < obj.schedule

  def __str__(self) -> str:
    return f'Task(room={self.room}, medicine={self.medicine}, patient_id={self.patient_id}, schedule={self.schedule})'

class Environment:
  def __init__(self, hospital_map: Dict[str, List[str]], tasks: List[Task]):
    self.hospital_map = hospital_map
    self.tasks = sorted(tasks) # tasks are ordered based on schedule
    self.medicine_storage: Dict[str, int] = {"Paracetamol": 10, "Ibuprofen": 5, "Antibiotic": 3}
    self.patients = set([101, 102, 103])

  def get_task(self) -> Optional[Task]:
    return self.tasks.pop(0) if self.tasks else None
  
  def get_neighbors(self, location: str) -> Optional[List[str]]:
    return self.hospital_map.get(location, None)

  def verify_patient_id(self, patient_id: int) -> bool:
    return patient_id in self.patients

  def get_medicine(self, medicine: str) -> bool:
    if self.medicine_storage.get(medicine, 0) > 0:
      self.medicine_storage[medicine] -= 1
      return True
    return False

  def alert_staff(self, location: str):
    print(f"STAFF ALERTED AT: {location}.")


class Agent:
  def __init__(self, env: Environment):
    self.env = env
    self.current_location: str = 'medicine_storage'
    self.inventory: Set[str] = set()

  def stow_medicine(self, medicine: str) -> bool:
    # if medicine is available, return okay status
    if self.env.get_medicine(medicine):
      self.inventory.add(medicine)
      return True
    return False
  
  def move_to(self, destination: str) -> bool:
    visited: Set[str] = set()
    path: List[str] = []
    def find_path(source: str) -> bool:
      visited.add(source)
      path.append(source)
      if source == destination:
        return True
      for room in self.env.hospital_map[source]:
        if room in visited:
          continue
        if find_path(room):
          return True
      path.pop()
      return False
    
    status = find_path(self.current_location)
    if status:
      self.current_location = destination
      print('Moved through:', *path)
    else:
      print('No path available')
    return status

  def deliver(self, task: Task):
    print(f'Completed Task: {task}')
    self.inventory.remove(task.medicine)

  def run(self):
    while True:
      task: Optional[Task] = self.env.get_task()
      if not task: # if no task
        print('All tasks completed')
        break

      print(f'Starting task: {task}')

      # if not at medical storage, go there
      if self.current_location != 'medicine_storage':
        if not self.move_to('medicine_storage'): # robot cant move to destination
          continue

      # if no medicine, alert staff
      # realistically, this should either create logs or keep alerting until the issue
      # is addressed. tho for the sake of simplicity, i've made it skip the current task
      if not self.stow_medicine(task.medicine):
        self.env.alert_staff(self.current_location)
        continue

      if not self.move_to(task.room): # robot can't move to destination
        continue
      # if patient id fails to verify, alert staff
      if not self.env.verify_patient_id(task.patient_id):
        self.env.alert_staff(self.current_location)
        continue
      
      self.deliver(task)


def run_agent():
  hospital_map = {
    'medicine_storage': ['corridor'],
    'room_101': ['corridor'],
    'room_102': ['corridor'],
    'room_103': ['corridor'],
    'nurses_station': ['corridor'],
    'corridor': ['medicine_storage', 'room_101', 'room_102', 'room_103', 'nurses_station']
  }

  # tasks are given with schedule in form of numbers
  tasks = [
    Task(room='room_101', medicine="Paracetamol", patient_id=101, schedule=1),
    Task(room='room_102', medicine="Ibuprofen", patient_id=102, schedule=3),
    Task(room='room_103', medicine="Antibiotic", patient_id=104, schedule=2)
  ]

  agent = Agent(Environment(hospital_map, tasks))
  agent.run()

run_agent()

Starting task: Task(room=room_101, medicine=Paracetamol, patient_id=101, schedule=1)
Moved through: medicine_storage corridor room_101
Completed Task: Task(room=room_101, medicine=Paracetamol, patient_id=101, schedule=1)
Starting task: Task(room=room_103, medicine=Antibiotic, patient_id=104, schedule=2)
Moved through: room_101 corridor medicine_storage
Moved through: medicine_storage corridor room_103
STAFF ALERTED AT: room_103.
Starting task: Task(room=room_102, medicine=Ibuprofen, patient_id=102, schedule=3)
Moved through: room_103 corridor medicine_storage
Moved through: medicine_storage corridor room_102
Completed Task: Task(room=room_102, medicine=Ibuprofen, patient_id=102, schedule=3)
All tasks completed


### Task 6

In [74]:
from typing import Tuple

class Environment:
  def __init__(self, initial_grid: List[List[Tuple[str, bool]]]):
    self.__grid = initial_grid

  def display_grid(self):
    print(*map(lambda row: ' '.join(map(lambda cell: '!' if cell[1] else '.', row)), self.__grid), sep='\n')

  def find_pos(self, room: str) -> Optional[Tuple[int, int]]:
    for i in range(len(self.__grid)):
      for j in range(len(self.__grid[0])):
        if self.__grid[i][j][0] != room:
          continue
        return (i, j)

  def is_on_fire(self, x: int, y: int):
    return self.__grid[x][y][1]
    
  def put_out_fire(self, x: int, y: int):
    self.__grid[x][y] = (self.__grid[x][y], False)

class Agent:
  def __init__(self, env: Environment, path: List[str]):
    self.env = env
    self.path = path
    self.current_idx: int = 0

  def can_move(self) -> bool:
    return self.current_idx < len(self.path)

  def run(self):
    current_room = self.path[self.current_idx]
    x, y = self.env.find_pos(current_room)
    print(f'Current Location: ({x}, {y}) = {current_room}')
    if self.env.is_on_fire(x, y):
      print(f'Detected fire at room: {current_room}')
      self.env.put_out_fire(x, y)
      print('Fire has been put out')
    self.current_idx += 1
    self.env.display_grid()

def run_agent():
  grid = [
    [('a', False), ('b', False), ('c', True)],
    [('d', False), ('e', True), ('f', False)],
    [('g', False), ('h', False), ('j', True)]
  ]
  env = Environment(grid)
  agent = Agent(env=env, path=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'j'])

  while agent.can_move():
    agent.run()
    print()

run_agent()

Current Location: (0, 0) = a
. . !
. ! .
. . !

Current Location: (0, 1) = b
. . !
. ! .
. . !

Current Location: (0, 2) = c
Detected fire at room: c
Fire has been put out
. . .
. ! .
. . !

Current Location: (1, 0) = d
. . .
. ! .
. . !

Current Location: (1, 1) = e
Detected fire at room: e
Fire has been put out
. . .
. . .
. . !

Current Location: (1, 2) = f
. . .
. . .
. . !

Current Location: (2, 0) = g
. . .
. . .
. . !

Current Location: (2, 1) = h
. . .
. . .
. . !

Current Location: (2, 2) = j
Detected fire at room: j
Fire has been put out
. . .
. . .
. . .

